## Create .yaml file to get the missing data

In [8]:
import os
import numpy as np
import time
import pandas as pd
from pathlib import Path

BASE_PATH = "C:\\Users\\MHJGuillermo\\PycharmProjects\\Doctorado\\Ising\\Varios_vecinos\\venv"
DATA_PATH = Path(os.path.join(BASE_PATH,"data/")).resolve()

### Get model to extrapol values

In [32]:
def model_function(x: float,a1,a2,a3):
    return 1/np.sqrt(-np.log(a1+a2*x+a3*x**2))

def get_extrapol_function(model_function,a1,a2,a3):
    def extrapol_funct(x):
        return model_function(x,a1,a2,a3)
    return extrapol_funct

def extrapol_funct_p1(x):
    a1 = 0.8941331066741727
    a2 = 0.8347580777184751
    return a1*np.sqrt(-np.log(1-a2*x))    


In [9]:
file_params = "inv_Tc_vs_inv_nv_sqrt_and_log_3_parms_params_curves.csv"
df_params = pd.read_csv(os.path.join(DATA_PATH,"Extrapolation",file_params),index_col=0)

In [33]:
list_alpha = [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]
min_nv = 16
max_nv = 25
for alpha in list_alpha:
    path_file = os.path.join(DATA_PATH,"h-0\\alpha_{}".format(alpha))
    try:
        os.mkdir(path_file)
    except FileExistsError:
        print('The directory {} has been created before'.format(path_file))
    for nv in range(min_nv,max_nv+1,1):
        if alpha == 1.0:
            temp_val_extrapol = 1/extrapol_funct_p1(1/nv)
        else:            
            # get params
            column = "P" + str(alpha)
            (a1,a2,a3) = df_params[column].values
            # get model
            extrapol_funct = get_extrapol_function(model_function, a1, a2, a3)
            temp_val_extrapol = extrapol_funct(1/nv)
        
        incert = 1/32
        num_samples = 32
        parallel = True
        if nv <= 20 : 
            num_samples = 256
            incert = 1/128
        elif (nv <= 22 and nv > 20):
            num_samples = 128
            incert = 1/64
        elif nv == 23:
            num_samples = 64
        else:
            num_samples = 32
            
        min_temp = temp_val_extrapol - (1/2)*(incert*num_samples)
        max_temp = temp_val_extrapol + (1/2)*(incert*num_samples)
        file_name = "alpha-{}_nv-{}_h-0.yml".format(alpha,nv)
#         if os.path.exists(os.path.join(path_file,file_name)):
#             print("File {} already exists.".format(file_name))
#         else:
        f = open(os.path.join(path_file,file_name),"w+")
        f.write("# Arbitrary metadata\n")
        f.write("metadata:\n  description: Ising chain with alpha={} and {} nearest neighbors\n".format(alpha,nv))
        listJ=1/np.arange(1,nv+1,1)**alpha
        f.write("\nsystem:\n  params:\n    interactions: {}\n".format(listJ.tolist()))
        f.write("\n    temperature:\n      min: {}\n      max: {}\n      num_samples: {}\n".format(min_temp,max_temp,num_samples))
        f.write("\n    magnetic_field: 0\n")
        f.write("\nexec:\n  parallel: {}".format(parallel))
        f.close()
print('Sucess generate all files')

The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.0 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.1 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.2 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.3 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.4 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.5 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\venv\data\h-0\alpha_1.6 has been created before
The directory C:\Users\MHJGuillermo\PycharmProjects\Doctorado\Ising\Varios_vecinos\

In [23]:
print( not os.path.exists(os.path.join(path_file,file_name)))

True


In [7]:
listJ = 1/np.arange(1,20+1,1)**1.1
listJ
print(listJ.tolist())
#print("[{}]".format(','.join([num for num in listJ])))

[1.0, 0.4665164957684037, 0.29865281994692067, 0.21763764082403103, 0.1702679845041569, 0.13932646701298948, 0.11759589342757772, 0.10153154954452942, 0.08919350686224783, 0.07943282347242814, 0.07152667656334291, 0.06499809515869194, 0.059519916389315516, 0.05486042411858822, 0.05085101371884506, 0.04736614270344992, 0.044310452642649344, 0.04161024226667093, 0.03920767043037826, 0.03705672245534738]


In [8]:
print([num for num in listJ])

[1.0, 0.4665164957684037, 0.29865281994692067, 0.21763764082403103, 0.1702679845041569, 0.13932646701298948, 0.11759589342757772, 0.10153154954452942, 0.08919350686224783, 0.07943282347242814, 0.07152667656334291, 0.06499809515869194, 0.059519916389315516, 0.05486042411858822, 0.05085101371884506, 0.04736614270344992, 0.044310452642649344, 0.04161024226667093, 0.03920767043037826, 0.03705672245534738]
